# DATA INGESTION
**Sources* :
- google trends

In [1]:
from pytrends.request import TrendReq

from datetime import date
import pandas as pd
import time

# Google Trends

## KEYWORD LIST

In [2]:
kw_list = ["bitcoin", "btc", "cryptocurrency"] 

## GET THE DATASET

In [3]:
pytrend = TrendReq(hl='en-US', tz=0)

In [4]:
pytrend.build_payload(kw_list, timeframe='2014-09-17 2021-10-20', geo='US')
google_df = pytrend.interest_over_time()
google_df

,bitcoin,btc,cryptocurrency,isPartial
date,,,,
2014-10-01,3,1,0,False
2014-11-01,3,1,0,False
2014-12-01,3,1,0,False
2015-01-01,3,1,0,False
2015-02-01,3,1,0,False
...,...,...,...,...
2021-06-01,32,5,4,False
2021-07-01,22,4,3,False
2021-08-01,21,4,3,False


#### The Limitations
 	--- Last 7 days will have hourly search trends \
 	--- daily data is only provided for query period shorter than 9 months and up to 36 hours before your search \
 	--- Between 9 month and 5 years, and any query longer than 5 years will only return monthly data

In [5]:
today = date.today().strftime("%Y-%m-%d")
today   # ajouter un mois

'2021-10-24'

In [6]:
histo = pd.interval_range(start=pd.Timestamp('2014-08-31'), end=pd.Timestamp(today), freq = '5M', closed = 'both')

In [7]:
for i in histo:
    print(i.left.date(), i.right.date())


2014-08-31 2015-01-31
2015-01-31 2015-06-30
2015-06-30 2015-11-30
2015-11-30 2016-04-30
2016-04-30 2016-09-30
2016-09-30 2017-02-28
2017-02-28 2017-07-31
2017-07-31 2017-12-31
2017-12-31 2018-05-31
2018-05-31 2018-10-31
2018-10-31 2019-03-31
2019-03-31 2019-08-31
2019-08-31 2020-01-31
2020-01-31 2020-06-30
2020-06-30 2020-11-30
2020-11-30 2021-04-30
2021-04-30 2021-09-30


In [8]:
df = []

for i in histo:
    pytrend.build_payload(kw_list, timeframe='{} {}'.format(i.left.date(), i.right.date()), geo='')
    google_df = pytrend.interest_over_time()
    df.append(google_df)
    time.sleep(1)
dfs = pd.concat(df)

In [9]:
dfs

,bitcoin,btc,cryptocurrency,isPartial
date,,,,
2014-08-31,20,6,0,False
2014-09-01,25,6,0,False
2014-09-02,24,4,0,False
2014-09-03,17,5,0,False
2014-09-04,20,6,0,False
...,...,...,...,...
2021-09-26,15,2,3,False
2021-09-27,16,3,2,False
2021-09-28,16,3,2,False


In [10]:
# ajouter le manque entre le derniere et aujourdui

In [11]:
new_start = (dfs.index.max() + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
new_start


'2021-10-01'

In [12]:
pytrend.build_payload(kw_list, timeframe= '{} {}'.format(new_start, today), geo='')
gap_df = pytrend.interest_over_time()


In [13]:
gap_df

,bitcoin,btc,cryptocurrency,isPartial
date,,,,
2021-10-01,43,9,4,False
2021-10-02,33,7,4,False
2021-10-03,31,6,4,False
2021-10-04,37,7,6,False
2021-10-05,44,9,6,False
2021-10-06,58,11,14,False
2021-10-07,49,10,14,False
2021-10-08,40,9,11,False
2021-10-09,40,6,7,False


In [14]:
gogole_trends_df = pd.concat([dfs, gap_df])

In [15]:
gogole_trends_df

,bitcoin,btc,cryptocurrency,isPartial
date,,,,
2014-08-31,20,6,0,False
2014-09-01,25,6,0,False
2014-09-02,24,4,0,False
2014-09-03,17,5,0,False
2014-09-04,20,6,0,False
...,...,...,...,...
2021-10-17,49,10,6,False
2021-10-18,62,12,12,False
2021-10-19,79,14,8,False


## EXTERNAL STORAGE

In [16]:
#gogole_trends_df.to_parquet('/home/giujorge/datalake/lab/Crypto/crypto/data/external/google/google_trends/btc_googletrends_{}.parquet.gzip'.format(date.today().strftime("%Y_%m_%d")), compression='gzip')